In [1]:
from pyomo.environ import *
import numpy as np
import qiskit
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.primitives import Sampler
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA

### Problema: 

O problema envolve a seleção de projetos em três anos, com o objetivo de maximizar o retorno financeiro dos projetos escolhidos, sob a condição de que os desembolsos anuais não ultrapassem os fundos disponíveis para cada ano.

$$\text{Maximize } z = 20x_1 + 40x_2 + 20x_3 + 15x_4 + 30x_5$$

#### Restrições

Ano 1: $ 5x_1 + 4x_2 + 3x_3 + 7x_4 + 8x_5 \leq 25$

Ano 2: $ 5x_1 + 7x_2 + 9x_3 + 4x_4 + 6x_5 \leq 25$

Ano 3: $ 8x_1 + 10x_2 + 2x_3 + 3x_4 + 10x_5 \leq 25$



In [2]:
project_selection_model = ConcreteModel()

#Problem data definition:
projects = [1, 2, 3, 4, 5]
returns = {1: 20, 2: 40, 3: 20, 4: 15, 5: 30}
cost = {
    1: [5, 5, 8],
    2: [4, 7, 10],
    3: [3, 9, 2],
    4: [7, 4, 3],
    5: [8, 6, 10]
}

annual_budget = [25, 25, 25]

#Decision variables
project_selection_model.x = Var(projects, within=Binary)

#OBJECTIVE FUNCTION: maximize total return
project_selection_model.obj = Objective(
    expr=sum(returns[j] * project_selection_model.x[j] for j in projects),
    sense=maximize
)


In [3]:
#Applying the restrictions
project_selection_model.ano1_constraint = Constraint(
    expr=sum(cost[j][0] * project_selection_model.x[j] for j in projects) <= annual_budget[0]
)
project_selection_model.ano2_constraint = Constraint(
    expr=sum(cost[j][1] * project_selection_model.x[j] for j in projects) <= annual_budget[1]
)
project_selection_model.y3_constraint = Constraint(
    expr=sum(cost[j][2] * project_selection_model.x[j] for j in projects) <= annual_budget[2]
)

In [4]:
solver = SolverFactory('glpk')
result = solver.solve(project_selection_model)

print("Optimization Status:", result.solver.status)
print("Solution Status:", result.solver.termination_condition)

print("\nSelected Projects:")
for j in projects:
    if project_selection_model.x[j].value == 1:
        print(f"Project {j}: Selected")

print("\nTotal Return:", project_selection_model.obj())

Optimization Status: ok
Solution Status: optimal

Selected Projects:
Project 1: Selected
Project 2: Selected
Project 3: Selected
Project 4: Selected

Total Return: 95.0


### Transcrevendo o problema para o tipo QUBO:

$$
\min_{x \in \{0,1\}^n} -\mu^T x + \alpha (1^T x - B)^2
$$

onde, 

- $x \in \{0,1\}^n$ vetor de variáveis binárias que indica quais projetos são selecionados $(x_j = 1)$ e quais não são $(x_j = 0)$.

- $\mu \in \mathbb{R}^n$ define os retornos esperados dos projetos.

- $ \alpha (1^T x - B)^2$ é uma penalidade que força a seleção de exatamente 
$B$ projetos. 

- $\alpha > 0$ é um peso que impõe a penalidade para garantir que exatamente $B$ projetos sejam selecionados.


In [28]:
mu = [20, 40, 20, 15, 30] # Expected returns from each project
alpha = 1 # Penalty 
n = len(mu) # Number of projects

In [29]:
cov_matrix = np.zeros((n, n))
print(cov_matrix)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [30]:
projectSelectionQP = QuadraticProgram("Project Selection")

print(projectSelectionQP.prettyprint())

Problem name: Project Selection

Minimize
  0

Subject to
  No constraints

  No variables



In [31]:
for i in range(n):
    projectSelectionQP.binary_var(name=f'x{i}')

In [32]:
linear = [-mu[i] for i in range(n)]
linear

[-20, -40, -20, -15, -30]

In [33]:
quadratic = {(f'x{i}', f'x{j}'): 0 for i in range(n) for j in range(i, n)}
quadratic

{('x0', 'x0'): 0,
 ('x0', 'x1'): 0,
 ('x0', 'x2'): 0,
 ('x0', 'x3'): 0,
 ('x0', 'x4'): 0,
 ('x1', 'x1'): 0,
 ('x1', 'x2'): 0,
 ('x1', 'x3'): 0,
 ('x1', 'x4'): 0,
 ('x2', 'x2'): 0,
 ('x2', 'x3'): 0,
 ('x2', 'x4'): 0,
 ('x3', 'x3'): 0,
 ('x3', 'x4'): 0,
 ('x4', 'x4'): 0}

In [34]:
projectSelectionQP.minimize(constant=0, linear=dict(zip([f'x{i}' for i in range(n)], linear)), quadratic=quadratic)

projectSelectionQP.linear_constraint(
    linear={'x0': 5, 'x1': 4, 'x2': 3, 'x3': 7, 'x4': 8},
    sense='<=',
    rhs=25
)

projectSelectionQP.linear_constraint(
    linear={'x0': 5, 'x1': 7, 'x2': 9, 'x3': 4, 'x4': 6},
    sense='<=',
    rhs=25
)

projectSelectionQP.linear_constraint(
    linear={'x0': 8, 'x1': 10, 'x2': 2, 'x3': 3, 'x4': 10},
    sense='<=',
    rhs=25
)

print(projectSelectionQP.prettyprint())

Problem name: Project Selection

Minimize
  -20*x0 - 40*x1 - 20*x2 - 15*x3 - 30*x4

Subject to
  Linear constraints (3)
    5*x0 + 4*x1 + 3*x2 + 7*x3 + 8*x4 <= 25  'c0'
    5*x0 + 7*x1 + 9*x2 + 4*x3 + 6*x4 <= 25  'c1'
    8*x0 + 10*x1 + 2*x2 + 3*x3 + 10*x4 <= 25  'c2'

  Binary variables (5)
    x0 x1 x2 x3 x4



In [35]:
algorithm_globals.random_seed = 10598

qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])
exact_mes = NumPyMinimumEigensolver()

/tmp/ipykernel_14710/1337795679.py:3: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.0, 0.0])


In [36]:
exact_optimizer = MinimumEigenOptimizer(exact_mes)
exact_result = exact_optimizer.solve(projectSelectionQP)
print(exact_result.prettyprint())

objective function value: -95.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=0.0
status: SUCCESS


In [37]:
qaoa_result = exact_optimizer.solve(projectSelectionQP)
print(qaoa_result.prettyprint())

objective function value: -95.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=0.0
status: SUCCESS


### Hamiltonian and QAOA circuit

In [43]:
qubo = QuadraticProgramToQubo().convert(projectSelectionQP)
print(qubo.prettyprint())

Problem name: Project Selection

Minimize
  126*c0@int_slack@0^2 + 504*c0@int_slack@0*c0@int_slack@1
  + 1008*c0@int_slack@0*c0@int_slack@2 + 2016*c0@int_slack@0*c0@int_slack@3
  + 2520*c0@int_slack@0*c0@int_slack@4 + 504*c0@int_slack@1^2
  + 2016*c0@int_slack@1*c0@int_slack@2 + 4032*c0@int_slack@1*c0@int_slack@3
  + 5040*c0@int_slack@1*c0@int_slack@4 + 2016*c0@int_slack@2^2
  + 8064*c0@int_slack@2*c0@int_slack@3 + 10080*c0@int_slack@2*c0@int_slack@4
  + 8064*c0@int_slack@3^2 + 20160*c0@int_slack@3*c0@int_slack@4
  + 12600*c0@int_slack@4^2 + 126*c1@int_slack@0^2
  + 504*c1@int_slack@0*c1@int_slack@1 + 1008*c1@int_slack@0*c1@int_slack@2
  + 2016*c1@int_slack@0*c1@int_slack@3 + 2520*c1@int_slack@0*c1@int_slack@4
  + 504*c1@int_slack@1^2 + 2016*c1@int_slack@1*c1@int_slack@2
  + 4032*c1@int_slack@1*c1@int_slack@3 + 5040*c1@int_slack@1*c1@int_slack@4
  + 2016*c1@int_slack@2^2 + 8064*c1@int_slack@2*c1@int_slack@3
  + 10080*c1@int_slack@2*c1@int_slack@4 + 8064*c1@int_slack@3^2
  + 20160*c1@in